In [10]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


# MLP Single and Multi Output
=== Summary ===


               model      acc     bacc       f1


      multi_output:valence 0.789062 0.500000 0.441048


      multi_output:arousal 0.769531 0.505937 0.450983


      single:valence 0.789062 0.500000 0.441048


      single:arousal 0.769531 0.505937 0.450983

      

In [11]:

# --------------------
# Paths
# --------------------
base_path_dat = './datasets/DEAP/deap-dataset/data_preprocessed_python'  # s01.dat ... s32.dat
features_base = './datasets/DEAP/deap-dataset/extracted_features'        # per-channel CSVs

EEG_CH_NAMES = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1',
                'P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8',
                'FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
SUBJECTS = range(1, 33)
TRIALS_PER_SUBJ = 40

In [7]:
def load_subject_features(subj_id):
    """Concatenate alpha/beta/gamma for 32 channels -> (40, 96)"""
    per_ch = []
    for ch in EEG_CH_NAMES:
        csv_path = os.path.join(features_base, ch, f"s{subj_id:02d}_bandpower.csv")
        df = pd.read_csv(csv_path)
        # Expect columns: trial, alpha_power, beta_power, gamma_power
        per_ch.append(df[['alpha_power','beta_power','gamma_power']].values)  # (40,3)
    X_subj = np.hstack(per_ch)  # (40, 32*3=96)
    return X_subj

def load_subject_labels(subj_id):
    """Load per-trial labels from sXX.dat -> (40,4): Valence,Arousal,Dominance,Liking"""
    with open(os.path.join(base_path_dat, f"s{subj_id:02d}.dat"), "rb") as f:
        raw = pickle.load(f, encoding="latin1")
    labels = raw['labels'].astype(np.float32)  # (40,4), values 1..9
    y_val = (labels[:, 0] > 5).astype(np.int32)  # Valence bin
    y_aro = (labels[:, 1] > 5).astype(np.int32)  # Arousal bin
    return y_val, y_aro

# --------------------
# Build X, y with guaranteed alignment
# --------------------
X_list, yv_list, ya_list = [], [], []
for s in SUBJECTS:
    Xs = load_subject_features(s)                # (40,96)
    yv_s, ya_s = load_subject_labels(s)          # (40,), (40,)
    assert Xs.shape[0] == TRIALS_PER_SUBJ == yv_s.shape[0] == ya_s.shape[0]
    X_list.append(Xs)
    yv_list.append(yv_s)
    ya_list.append(ya_s)

X = np.vstack(X_list)            # (32*40, 96) = (1280, 96)
y_valence = np.hstack(yv_list)   # (1280,)
y_arousal = np.hstack(ya_list)   # (1280,)

print("X shape:", X.shape, " y_valence:", y_valence.shape, " y_arousal:", y_arousal.shape)

# --------------------
# Split + scale
# --------------------
X_train, X_test, yv_train, yv_test, ya_train, ya_test = train_test_split(
    X, y_valence, y_arousal, test_size=0.2, random_state=42, stratify=y_valence
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test,  dtype=torch.float32)
yv_train_tensor = torch.tensor(yv_train, dtype=torch.long)
yv_test_tensor  = torch.tensor(yv_test,  dtype=torch.long)
ya_train_tensor = torch.tensor(ya_train, dtype=torch.long)
ya_test_tensor  = torch.tensor(ya_test,  dtype=torch.long)

train_ds = TensorDataset(X_train_tensor, yv_train_tensor, ya_train_tensor)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

# --------------------
# Models
# --------------------
class MultiOutputMLP(nn.Module):
    def __init__(self, input_dim=96, h1=128, h2=64, p=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.do  = nn.Dropout(p)
        self.act = nn.ReLU()
        self.out_v = nn.Linear(h2, 2)
        self.out_a = nn.Linear(h2, 2)
    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.do(x)
        x = self.act(self.fc2(x))
        return self.out_v(x), self.out_a(x)

class SingleOutputMLP(nn.Module):
    def __init__(self, input_dim=96, h1=128, h2=64, p=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.do  = nn.Dropout(p)
        self.act = nn.ReLU()
        self.out = nn.Linear(h2, 2)
    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.do(x)
        x = self.act(self.fc2(x))
        return self.out(x)

def report(name, y_true, y_pred):
    acc  = accuracy_score(y_true, y_pred)
    bacc = balanced_accuracy_score(y_true, y_pred)
    f1   = f1_score(y_true, y_pred, average="macro")
    print(f"{name}: acc={acc:.3f}, bacc={bacc:.3f}, f1_macro={f1:.3f}")
    return acc, bacc, f1

device = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 20

# --------------------
# Train multi-output
# --------------------
model_multi = MultiOutputMLP().to(device)
crit = nn.CrossEntropyLoss()
opt  = optim.Adam(model_multi.parameters(), lr=1e-3)

for _ in range(EPOCHS):
    model_multi.train()
    for xb, yv, ya in train_loader:
        xb, yv, ya = xb.to(device), yv.to(device), ya.to(device)
        opt.zero_grad()
        out_v, out_a = model_multi(xb)
        loss = crit(out_v, yv) + crit(out_a, ya)
        loss.backward()
        opt.step()

model_multi.eval()
with torch.no_grad():
    out_v, out_a = model_multi(X_test_tensor.to(device))
    yv_pred = torch.argmax(out_v, 1).cpu().numpy()
    ya_pred = torch.argmax(out_a, 1).cpu().numpy()

print("=== Multi-output MLP ===")
mv = report("Valence", yv_test, yv_pred)
ma = report("Arousal", ya_test, ya_pred)

# --------------------
# Train & eval single-output models
# --------------------
def train_single(Xtr, ytr, Xte, yte, label_name):
    model = SingleOutputMLP().to(device)
    crit  = nn.CrossEntropyLoss()
    opt   = optim.Adam(model.parameters(), lr=1e-3)
    ds = TensorDataset(torch.tensor(Xtr, dtype=torch.float32),
                       torch.tensor(ytr, dtype=torch.long))
    dl = DataLoader(ds, batch_size=64, shuffle=True)
    for _ in range(EPOCHS):
        model.train()
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            logits = model(xb)
            loss = crit(logits, yb)
            loss.backward()
            opt.step()
    model.eval()
    with torch.no_grad():
        logits = model(torch.tensor(Xte, dtype=torch.float32).to(device))
        y_pred = torch.argmax(logits, 1).cpu().numpy()
    print(f"=== Single-output MLP — {label_name} ===")
    return report(label_name, yte, y_pred)

sv = train_single(X_train, yv_train, X_test, yv_test, "Valence")
sa = train_single(X_train, ya_train, X_test, ya_test, "Arousal")

summary = pd.DataFrame({
    "model": ["multi_output:valence","multi_output:arousal","single:valence","single:arousal"],
    "acc":   [mv[0], ma[0], sv[0], sa[0]],
    "bacc":  [mv[1], ma[1], sv[1], sa[1]],
    "f1":    [mv[2], ma[2], sv[2], sa[2]],
})
print("\n=== Summary ===")
print(summary.to_string(index=False))


X shape: (1280, 96)  y_valence: (1280,)  y_arousal: (1280,)
=== Multi-output MLP ===
Valence: acc=0.789, bacc=0.500, f1_macro=0.441
Arousal: acc=0.770, bacc=0.506, f1_macro=0.451
=== Single-output MLP — Valence ===
Valence: acc=0.789, bacc=0.500, f1_macro=0.441
=== Single-output MLP — Arousal ===
Arousal: acc=0.770, bacc=0.506, f1_macro=0.451

=== Summary ===
               model      acc     bacc       f1
multi_output:valence 0.789062 0.500000 0.441048
multi_output:arousal 0.769531 0.505937 0.450983
      single:valence 0.789062 0.500000 0.441048
      single:arousal 0.769531 0.505937 0.450983


## MLP with 9 categorical outputs (Not Binary)

=== Summary ===
               
               model      acc     bacc       f1


    multi_output:valence 0.121094 0.130769 0.102354


    multi_output:arousal 0.132812 0.203067 0.120390


      single:valence 0.183594 0.140942 0.126499

      
      single:arousal 0.175781 0.251657 0.168949

In [12]:

N_CLASSES = 9  # categories 1..9

def load_subject_features(subj_id):
    """Concatenate alpha/beta/gamma for 32 channels -> (40, 96)"""
    per_ch = []
    for ch in EEG_CH_NAMES:
        csv_path = os.path.join(features_base, ch, f"s{subj_id:02d}_bandpower.csv")
        df = pd.read_csv(csv_path)
        per_ch.append(df[['alpha_power','beta_power','gamma_power']].values)  # (40,3)
    X_subj = np.hstack(per_ch)  # (40, 96)
    return X_subj

def load_subject_labels_rounded(subj_id):
    """
    Load continuous labels (1..9) and round to nearest int in [1..9],
    returning class indices in [0..8] (for PyTorch CrossEntropyLoss).
    """
    with open(os.path.join(base_path_dat, f"s{subj_id:02d}.dat"), "rb") as f:
        raw = pickle.load(f, encoding="latin1")
    labels = raw['labels'].astype(np.float32)  # (40,4)
    v = labels[:, 0]  # valence
    a = labels[:, 1]  # arousal
    # Round to 1..9 categories, then map to 0..8
    v_cat = np.clip(np.rint(v).astype(int), 1, 9) - 1
    a_cat = np.clip(np.rint(a).astype(int), 1, 9) - 1
    return v_cat, a_cat  # (40,), (40,)

# --------------------
# Build X, y with guaranteed alignment
# --------------------
X_list, yv_list, ya_list = [], [], []
for s in SUBJECTS:
    Xs = load_subject_features(s)                  # (40,96)
    yv_s, ya_s = load_subject_labels_rounded(s)    # (40,), (40,) in 0..8
    assert Xs.shape[0] == TRIALS_PER_SUBJ == yv_s.shape[0] == ya_s.shape[0]
    X_list.append(Xs)
    yv_list.append(yv_s)
    ya_list.append(ya_s)

X = np.vstack(X_list)                   # (1280, 96)
y_valence = np.hstack(yv_list)          # (1280,) 0..8
y_arousal = np.hstack(ya_list)          # (1280,) 0..8

print("X shape:", X.shape)
print("Valence class counts (1..9):", np.bincount(y_valence, minlength=N_CLASSES))
print("Arousal class counts (1..9):", np.bincount(y_arousal, minlength=N_CLASSES))

# --------------------
# Split + scale
# --------------------
# Stratify on valence (you can stratify on arousal instead; for a stricter setup use GroupKFold by subject)
X_train, X_test, yv_train, yv_test, ya_train, ya_test = train_test_split(
    X, y_valence, y_arousal, test_size=0.2, random_state=42, stratify=y_valence
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test,  dtype=torch.float32)
yv_train_tensor = torch.tensor(yv_train, dtype=torch.long)
yv_test_tensor  = torch.tensor(yv_test,  dtype=torch.long)
ya_train_tensor = torch.tensor(ya_train, dtype=torch.long)
ya_test_tensor  = torch.tensor(ya_test,  dtype=torch.long)

train_ds = TensorDataset(X_train_tensor, yv_train_tensor, ya_train_tensor)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

# --------------------
# Optional: class weights (handle rare categories better)
# --------------------
def make_class_weights(y, n_classes=N_CLASSES):
    counts = np.bincount(y, minlength=n_classes).astype(np.float32)
    # inverse frequency; avoid division by zero
    w = 1.0 / np.maximum(counts, 1.0)
    w = w * (n_classes / w.sum())  # normalize for stability
    return torch.tensor(w, dtype=torch.float32)

w_val = make_class_weights(yv_train)
w_aro = make_class_weights(ya_train)

# --------------------
# Models
# --------------------
class MultiOutputMLP(nn.Module):
    def __init__(self, input_dim=96, h1=256, h2=128, p=0.3, n_classes=N_CLASSES):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.do  = nn.Dropout(p)
        self.act = nn.ReLU()
        self.out_v = nn.Linear(h2, n_classes)
        self.out_a = nn.Linear(h2, n_classes)
    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.do(x)
        x = self.act(self.fc2(x))
        return self.out_v(x), self.out_a(x)

class SingleOutputMLP(nn.Module):
    def __init__(self, input_dim=96, h1=256, h2=128, p=0.3, n_classes=N_CLASSES):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.do  = nn.Dropout(p)
        self.act = nn.ReLU()
        self.out = nn.Linear(h2, n_classes)
    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.do(x)
        x = self.act(self.fc2(x))
        return self.out(x)

def report(name, y_true, y_pred):
    acc  = accuracy_score(y_true, y_pred)
    bacc = balanced_accuracy_score(y_true, y_pred)
    f1   = f1_score(y_true, y_pred, average="macro")
    print(f"{name}: acc={acc:.3f}, bacc={bacc:.3f}, f1_macro={f1:.3f}")
    return acc, bacc, f1

device = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 40

# --------------------
# Train multi-output (two 9-way heads)
# --------------------
model_multi = MultiOutputMLP().to(device)
crit_v = nn.CrossEntropyLoss(weight=w_val.to(device))
crit_a = nn.CrossEntropyLoss(weight=w_aro.to(device))
opt  = optim.Adam(model_multi.parameters(), lr=1e-3, weight_decay=1e-5)

for _ in range(EPOCHS):
    model_multi.train()
    for xb, yv, ya in train_loader:
        xb, yv, ya = xb.to(device), yv.to(device), ya.to(device)
        opt.zero_grad()
        out_v, out_a = model_multi(xb)
        loss = crit_v(out_v, yv) + crit_a(out_a, ya)
        loss.backward()
        opt.step()

model_multi.eval()
with torch.no_grad():
    out_v, out_a = model_multi(X_test_tensor.to(device))
    yv_pred = torch.argmax(out_v, 1).cpu().numpy()
    ya_pred = torch.argmax(out_a, 1).cpu().numpy()

print("=== Multi-output MLP (9-class) ===")
mv = report("Valence", yv_test, yv_pred)
ma = report("Arousal", ya_test, ya_pred)

# --------------------
# Train & eval separate MLPs (9-class each)
# --------------------
def train_single(Xtr, ytr, Xte, yte, label_name, w_cls):
    model = SingleOutputMLP().to(device)
    crit  = nn.CrossEntropyLoss(weight=w_cls.to(device))
    opt   = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    ds = TensorDataset(torch.tensor(Xtr, dtype=torch.float32),
                       torch.tensor(ytr, dtype=torch.long))
    dl = DataLoader(ds, batch_size=64, shuffle=True)
    for _ in range(EPOCHS):
        model.train()
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            logits = model(xb)
            loss = crit(logits, yb)
            loss.backward()
            opt.step()
    model.eval()
    with torch.no_grad():
        logits = model(torch.tensor(Xte, dtype=torch.float32).to(device))
        y_pred = torch.argmax(logits, 1).cpu().numpy()
    print(f"=== Single-output MLP (9-class) — {label_name} ===")
    return report(label_name, yte, y_pred)

sv = train_single(X_train, yv_train, X_test, yv_test, "Valence", w_val)
sa = train_single(X_train, ya_train, X_test, ya_test, "Arousal", w_aro)

summary = pd.DataFrame({
    "model": ["multi_output:valence","multi_output:arousal","single:valence","single:arousal"],
    "acc":   [mv[0], ma[0], sv[0], sa[0]],
    "bacc":  [mv[1], ma[1], sv[1], sa[1]],
    "f1":    [mv[2], ma[2], sv[2], sa[2]],
})
print("\n=== Summary ===")
print(summary.to_string(index=False))


X shape: (1280, 96)
Valence class counts (1..9): [221 249 217 224 173  66  88  36   6]
Arousal class counts (1..9): [141 238 284 240 116 124  79  39  19]
=== Multi-output MLP (9-class) ===
Valence: acc=0.168, bacc=0.173, f1_macro=0.144
Arousal: acc=0.168, bacc=0.232, f1_macro=0.153
=== Single-output MLP (9-class) — Valence ===
Valence: acc=0.156, bacc=0.152, f1_macro=0.128
=== Single-output MLP (9-class) — Arousal ===
Arousal: acc=0.180, bacc=0.224, f1_macro=0.154

=== Summary ===
               model      acc     bacc       f1
multi_output:valence 0.167969 0.173329 0.143822
multi_output:arousal 0.167969 0.232476 0.153016
      single:valence 0.156250 0.152031 0.127518
      single:arousal 0.179688 0.223724 0.154343
